In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval
from bisect import bisect_left, bisect_right
from src.usa.plot_utils import plot_new_facilities, plot_state, plot_blockgroups
from src.usa.utils import compute_medical_deserts
from src.facility_location.utils import compute_minimum_distances
from src.usa.states import USAState
import plotly.graph_objects as go
import os


In [2]:
os.chdir('..')

In [15]:
facility_name = 'top_3_pharmacy_chains'
state_name = 'Mississippi'
State = USAState(state_name)
census_df = State.get_census_data(level='blockgroup')

In [16]:
df1 = pd.read_csv('data/usa/new_facilities/' + facility_name + '/new_facilities_1.csv', index_col=0)
df2 = pd.read_csv('data/usa/new_facilities/' + facility_name + '/new_facilities_2.csv', index_col=0)
df3 = pd.read_csv('data/usa/new_facilities/' + facility_name + '/new_facilities_inf.csv', index_col=0)

In [23]:
facilities = {}
facilities[1] = literal_eval(df1.loc[state_name]['100'])
facilities[2] = literal_eval(df2.loc[state_name]['100'])
facilities[3] = literal_eval(df3.loc[state_name]['100'])
all_facilities = set(facilities[1] + facilities[2] + facilities[3])

In [24]:
def get_facility_score(facility, facilities):
    thresholds = [5, 10, 25, 50, 100, np.inf]
    
    score = 0
    for i in range(1, 4):
        if facility in facilities[i]:
            id = facilities[i].index(facility)
            threshold = bisect_left(a=thresholds, x=id)
            score += np.sqrt(1/thresholds[threshold])
        else:
            score += 0
    
    return score


In [25]:
scores = {facility: get_facility_score(facility, facilities) for facility in all_facilities}

In [26]:
all_facilities = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1], reverse=True)}

In [32]:
def analyze_new_facilities(k, census_df, poverty_threshold=20, n_urban=2, n_rural=10):
    new_facilities = list(all_facilities.keys())[:k]
    census_df = compute_minimum_distances(census_df=census_df, facilities=new_facilities,
                                          existing_distances_label='closest_distance_' + facility_name,
                                          new_distances_label='temp_distance'
                                          )
    desert_df = compute_medical_deserts(state_df=census_df, poverty_threshold=20,
                                        n_urban=2, n_rural=10, distance_label='temp_distance')
    fig = go.Figure()
    fig, bounds = plot_state(fig, State)
    fig = plot_new_facilities(fig, list(all_facilities.keys())[:k], census_df)
    # fig = plot_blockgroups(fig, desert_df)
    
    return fig, len(desert_df)
    

In [33]:
fig, n_deserts = analyze_new_facilities(50, census_df)
print(n_deserts)
fig.show()

233
